In [119]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score


In [13]:
# from main import UserPredictor
# model = UserPredictor()


# train data
train_users = pd.read_csv("data/train_users.csv")
train_logs = pd.read_csv("data/train_logs.csv")
train_y = pd.read_csv("data/train_y.csv")
# model.fit(train_users, train_logs, train_y)


# test data
test_users = pd.read_csv("data/test1_users.csv")
test_logs = pd.read_csv("data/test1_logs.csv")
test_y = pd.read_csv("data/test1_y.csv")
# y_pred = model.predict(test_users, test_logs)




In [25]:
train_users

,user_id,names,age,past_purchase_amt,badge
0,0,Adriana Mcclure,26,39.344704,gold
1,1,Stacy Gilmore,67,15.840151,silver
2,2,Joanna Walsh,50,1099.420085,bronze
3,3,Eduardo Moore,65,5.880239,bronze
4,4,Angela Freeman,88,1312.296847,bronze
...,...,...,...,...,...
29995,29995,William Lewis,87,108.914205,bronze
29996,29996,James Miranda,49,291.969273,gold
29997,29997,Carlos Davis,46,2.609698,bronze
29998,29998,Makayla Smith,23,164.885894,bronze


In [152]:
class UserPredictor():
    def __init__(self):
      
        self.pipe = None
        self.xcol = None
        
    
    def concatFeature(self, users, log):
        time = log.groupby("user_id").sum()
        users["index"] = users["user_id"]
        new = pd.concat([users.set_index("index") ,time["seconds"]], axis=1).fillna(0)
        return new
    
    
    def fit(self, train_users, train_logs, train_y):
#         self.users = train_users.copy()
#         self.logs = train_logs.copy()
#         self.y_val = train_y.copy()
#         self.users['y'] = self.y_val['y']
        
#       add time feature to the dataframe
#         time = train_logs.groupby("user_id").sum()
#         new = pd.concat([self.users , time["seconds"]], axis=1).fillna(0)
        new = self.concatFeature(train_users,train_logs)
        
        self.xcol = ["user_id", "past_purchase_amt","age","seconds"]
        
        #model
        self.pipe = pipe = Pipeline([
            ("poly", PolynomialFeatures(degree=2)),
            ("std", StandardScaler()),
            ("lr", LogisticRegression(max_iter=100))
        ])
        self.pipe.fit(new[self.xcol],train_y['y'])
        
        # cross val debug
        scores = cross_val_score(self.pipe, new[self.xcol], train_y["y"])
        print(f"AVG: {scores.mean()}, STD: {scores.std()}\n")
            
    def score(self,test_users,test_logs,test_y):
        
#         add time
#         time = test_logs.groupby("user_id").sum()
#         test_users["index"] = test_users["user_id"]
#         new = pd.concat([test_users.set_index("index") ,time], axis=1).fillna(0)
        new = self.concatFeature(test_users,test_logs)
        
        # cross val debug
#         scores = cross_val_score(self.pipe, new[self.xcol], train_y["y"])
#         print(f"AVG: {scores.mean()}, STD: {scores.std()}\n")
        
    
        return self.pipe.score(new[self.xcol],test_y['y'])

    def predict(self, test_users, test_logs):
#         time = test_logs.groupby("user_id").sum()
#         test_users["index"] = test_users["user_id"]
#         new = pd.concat([test_users.set_index("index") ,time], axis=1).fillna(0)
        new = self.concatFeature(test_users,test_logs)
        return self.pipe.predict(new[self.xcol])
    

In [153]:
model = UserPredictor()
model.fit(train_users,train_logs, train_y)
model.score(test_users,test_logs,test_y)
y_pred = model.predict(test_users, test_logs)
y_pred[2]

AVG: 0.8900333333333332, STD: 0.003620006138730228



False

In [77]:
train_users

,user_id,names,age,past_purchase_amt,badge
0,0,Adriana Mcclure,26,39.344704,gold
1,1,Stacy Gilmore,67,15.840151,silver
2,2,Joanna Walsh,50,1099.420085,bronze
3,3,Eduardo Moore,65,5.880239,bronze
4,4,Angela Freeman,88,1312.296847,bronze
...,...,...,...,...,...
29995,29995,William Lewis,87,108.914205,bronze
29996,29996,James Miranda,49,291.969273,gold
29997,29997,Carlos Davis,46,2.609698,bronze
29998,29998,Makayla Smith,23,164.885894,bronze


In [83]:
time = train_logs.groupby("user_id").sum()
time.head()

new = pd.concat([train_users ,time["seconds"]], axis=1).fillna(0)
# new['seconds'] = new['seconds'].fillna(0)
new



,user_id,names,age,past_purchase_amt,badge,seconds
0,0,Adriana Mcclure,26,39.344704,gold,327.0
1,1,Stacy Gilmore,67,15.840151,silver,78.0
2,2,Joanna Walsh,50,1099.420085,bronze,432.0
3,3,Eduardo Moore,65,5.880239,bronze,0.0
4,4,Angela Freeman,88,1312.296847,bronze,0.0
...,...,...,...,...,...,...
29995,29995,William Lewis,87,108.914205,bronze,563.0
29996,29996,James Miranda,49,291.969273,gold,532.0
29997,29997,Carlos Davis,46,2.609698,bronze,0.0
29998,29998,Makayla Smith,23,164.885894,bronze,0.0
